# Text Summarization
## Find Text


In [4]:
import pysrt
import pandas as pd
import re
from sentence_transformers import SentenceTransformer
# install faiss-cpu
import faiss
import time
subs = pysrt.open("The.Batman.2022.1080p.WEB-DL.DDP5.1.Atmos.H.264-EVO-HI.srt")

DF = pd.DataFrame([
  {
    "Text": sub.text
} for sub in subs])

DF
def remove_noise(text):
    text = re.sub("<.*>", " ", text)
    text = re.sub("{.*}", " ", text)
    text = re.sub("\[.*\]", " ", text)
    text = text.strip()
    return text

DF['clean'] = DF['Text'].apply(remove_noise)

DF = DF[DF['clean'] != ""]

DF


,Text,clean
0,"<font color=""#ffff00"">Provided by explosivesku...",https://twitter.com/kaboomskull
11,[man] Hey!,Hey!
14,<i>Good evening and welcome to\nGC-1 News live...,Good evening and welcome to\nGC-1 News live at...
16,<i>just-released polls have\nincumbent Mayor D...,just-released polls have\nincumbent Mayor Don ...
17,<i>and 28-year-old grassroots challenger\nBell...,and 28-year-old grassroots challenger\nBella R...
...,...,...
2404,You should go.,You should go.
2405,Selina...,Selina...
2406,Take care of yourself.,Take care of yourself.
2414,"<font color=""#ffff00"">Provided by explosivesku...",https://twitter.com/kaboomskull


Create A Search Engine
Using each sentence as your “documents”, create a search engine to find specific pieces of text.


In [5]:
# this is creating the embeddings 
model = SentenceTransformer('msmarco-MiniLM-L-12-v3')
bodies_text_embds = model.encode(DF['clean'].to_list())
# Create an index using FAISS
index = faiss.IndexFlatL2(bodies_text_embds.shape[1])
index.add(bodies_text_embds)
faiss.write_index(index, 'index_bodies')

bodies_text_embds
# define a search 
def search(query, k):
    
    t=time.time()
    query_vector = model.encode([query])
    top_k = index.search(query_vector, k)
    print('totaltime: {}'.format(time.time()-t))
    return [DF['clean'].to_list()[_id] for _id in top_k[1].tolist()[0]]


C:\Users\user7\AppData\Roaming\Python\Python39\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user7\.cache\huggingface\hub\models--sentence-transformers--msmarco-MiniLM-L-12-v3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Search for several items.

In [6]:
search("cop", 10)
search("gun", 10)
search("car", 10)


totaltime: 0.0249330997467041
totaltime: 0.019948482513427734
totaltime: 0.02592778205871582


['Her body was in his car.',
 "Yeah, he's got a car.",
 'Get out of the car!',
 'What?',
 'What?',
 'What?',
 'What?',
 'What?',
 'Get out of the car and show your hands!',
 '-  \n-']